In [14]:
import pandas as pd 
import numpy as np
import scipy as sp 
from sklearn.metrics.pairwise import cosine_similarity
import operator
import fastparquet as fp 
import pyarrow as pa 
import pyarrow.parquet as pq

In [27]:
df_item_rec = pd.read_parquet('../data/processed/df_item_rec.parquet')
df_item_rec.head()

,title,genres
id,,
10.0,Counter-Strike,Action
20.0,Team Fortress Classic,Action
30.0,Day of Defeat,Action
40.0,Deathmatch Classic,Action
50.0,Half-Life: Opposing Force,Action


In [28]:
#Se crean los dummies 
df_item_rec = pd.get_dummies(df_item_rec, columns=['genres'], prefix='')

df_item_rec = df_item_rec.groupby(['id', 'title']).sum().reset_index()

df_item_rec.head()

,id,title,_Accounting,_Action,_Adventure,_Animation &amp; Modeling,_Audio Production,_Casual,_Design &amp; Illustration,_Early Access,...,_Photo Editing,_RPG,_Racing,_Simulation,_Software Training,_Sports,_Strategy,_Utilities,_Video Production,_Web Publishing
0,10.0,Counter-Strike,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20.0,Team Fortress Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30.0,Day of Defeat,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,40.0,Deathmatch Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,50.0,Half-Life: Opposing Force,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
similitudes = cosine_similarity(df_item_rec.iloc[:,3:])
print(similitudes)

[[1. 1. 1. ... 0. 1. 1.]
 [1. 1. 1. ... 0. 1. 1.]
 [1. 1. 1. ... 0. 1. 1.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [1. 1. 1. ... 0. 1. 1.]
 [1. 1. 1. ... 0. 1. 1.]]


In [31]:
similitudes.shape

(27422, 27422)

In [37]:
def recomendacion_juegos(id):
    
    id = int(id)
    # Filtrar el juego e igualarlo a  su ID
    juego_seleccionado = df_item_rec[df_item_rec['id'] == id]
    # devolver error en caso de vacio
    if juego_seleccionado.empty:
        return "El juego con el ID especificado no existe en la base de datos."
    
    # Calcula la similitud del juego que se ingresa con otros juegos del dataframe
    similarity_scores = similitudes[df_item_rec[df_item_rec['id'] == id].index[0]]
    
    # Calcula los índices de los juegos más similares (excluyendo el juego de entrada)
    indices_juegos_similares = similarity_scores.argsort()[::-1][1:6]
    
    # Obtener los nombres de los juegos 5 recomendados
    juegos_recomendados = df_item_rec.iloc[indices_juegos_similares]['title']
    
    return juegos_recomendados

In [38]:
recomendacion_juegos(772540)

12658                Tempest: Pirate Action RPG
12701                  GGXrd System Voice - MAY
12700             GGXrd System Voice - KY KISKE
12699           GGXrd System Voice - SOL BADGUY
12698    GGXrd Character Unlock - LEO WHITEFANG
Name: title, dtype: object

In [55]:
df_item_rec

,id,title,_Accounting,_Action,_Adventure,_Animation &amp; Modeling,_Audio Production,_Casual,_Design &amp; Illustration,_Early Access,...,_Photo Editing,_RPG,_Racing,_Simulation,_Software Training,_Sports,_Strategy,_Utilities,_Video Production,_Web Publishing
0,10.0,Counter-Strike,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20.0,Team Fortress Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30.0,Day of Defeat,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,40.0,Deathmatch Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,50.0,Half-Life: Opposing Force,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27417,901805.0,Saints Row: The Third Season Pass DLC Pack,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27418,2028055.0,Tom Clancy's Ghost Recon Future Soldier - Seas...,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27419,2028056.0,Worms Revolution Season Pass,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
27420,2028103.0,Assassin’s Creed® III Season Pass,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
#Cuento filas
cant_filas= len(df_item_rec)

#Calculo la mitad
mitad_filas= cant_filas // 10
 
#Selecciono la mitad superior
df_rec= df_item_rec.iloc[:mitad_filas]

In [58]:
df_rec.shape

(2742, 24)

In [59]:
similitudes_railway = cosine_similarity(df_rec.iloc[:,3:])

In [60]:
df_rec.to_parquet('../data/df_rec.parquet')

In [61]:
def encontrar_juegos_similares(id):
    # Encuentra el índice del juego ingresado por ID
    juego_indice = df_rec.index[df_rec['id'] == id].tolist()[0]
    
    # Extrae las características del juego ingresado
    juego_caracteristicas = df_rec.iloc[juego_indice, 3:].values.reshape(1, -1)
    
    # Calcula la similitud coseno entre el juego ingresado y todos los demás juegos
    similitudes_render = cosine_similarity(df_rec.iloc[:, 3:], juego_caracteristicas)
    
    # Obtiene los índices de los juegos más similares (excluyendo el juego de entrada)
    indices_juegos_similares = similitudes_render.argsort(axis=0)[::-1][1:6]
    indices_juegos_similares = indices_juegos_similares.flatten()[1:]
    
    # Obtiene los juegos más similares en función de los índices
    juegos_similares = df_rec.iloc[indices_juegos_similares]['title']
    
    return juegos_similares

In [62]:
encontrar_juegos_similares(3310)

808              Puzzler World
589                 Fluttabyes
590                  Yosumin!™
598    Puzzle Quest: Galactrix
Name: title, dtype: object

In [63]:
def recomendacion_juego(id: int):
    
    # Verificamos si el juego con game_id existe en df_games
    game = df_rec[df_rec['id'] == id]

    if game.empty:
        return("El juego '{id}' no posee registros.")
    
    # Obtenemos el índice del juego dado
    idx = game.index[0]

    # Tomamos una muestra aleatoria del DataFrame df_games
    sample_size = 2000  # Definimos el tamaño de la muestra (ajusta según sea necesario)
    df_sample = df_rec.sample(n=sample_size, random_state=42)  # Ajustamos la semilla aleatoria según sea necesario

    # Calculamos la similitud de contenido solo para el juego dado y la muestra
    sim_scores = cosine_similarity([df_rec.iloc[idx, 3:]], df_sample.iloc[:, 3:])

    # Obtenemos las puntuaciones de similitud del juego dado con otros juegos
    sim_scores = sim_scores[0]

    # Ordenamos los juegos por similitud en orden descendente
    similar_games = [(i, sim_scores[i]) for i in range(len(sim_scores)) if i != idx]
    similar_games = sorted(similar_games, key=lambda x: x[1], reverse=True)

    # Obtenemos los 5 juegos más similares
    similar_game_indices = [i[0] for i in similar_games[:5]]

    # Listamos los juegos similares (solo nombres)
    similar_game_names = df_sample['title'].iloc[similar_game_indices].tolist()

    return {"similar_games": similar_game_names}

In [64]:
recomendacion_juego(3310)

{'similar_games': ['Rocksmith - Foster the People - Pumped Up Kicks',
  'Rocksmith - Albert King with Stevie Ray Vaughan - Born Under a Bad Sign',
  'Chime',
  'Samantha Swift and the Hidden Roses of Athena',
  'Elizabeth Find M.D. - Diagnosis Mystery - Season 2']}